In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Set up directories for benign and malignant images
train_dir = 'C:/Users/Timothy/OneDrive/Desktop/train'
val_dir = 'C:/Users/Timothy/OneDrive/Desktop/val'

# Image data generators for loading and augmenting images
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize images
    rotation_range=30,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resizing images to fit the model
    batch_size=32,
    class_mode='binary'  # Binary classification (benign or malignant)
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[checkpoint]
)

# Save the final model
model.save('final_model.h5')


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


C:\Users\Timothy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# Path to your labeled dataset
DATASET_PATH = r"C:/Users/Timothy/OneDrive/Desktop/proj"  # Change this to your dataset path
IMAGE_SIZE = 256
BATCH_SIZE = 32
SEED = 42

# Load training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Build the CNN model
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)

# Save the model
model.save("breast_cancer_model.h5")
print("✅ Model saved as breast_cancer_model.h5")


Found 126 files belonging to 2 classes.
Using 101 files for training.
Found 126 files belonging to 2 classes.
Using 25 files for validation.
Epoch 1/10


C:\Users\Timothy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5622 - loss: 1.1666 - val_accuracy: 0.4400 - val_loss: 0.7219
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 949ms/step - accuracy: 0.4986 - loss: 0.7235 - val_accuracy: 0.4400 - val_loss: 0.7278
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 943ms/step - accuracy: 0.4368 - loss: 0.7161 - val_accuracy: 0.4400 - val_loss: 0.6976
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 940ms/step - accuracy: 0.4682 - loss: 0.6941 - val_accuracy: 0.4000 - val_loss: 0.6987
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 929ms/step - accuracy: 0.5078 - loss: 0.6949 - val_accuracy: 0.4400 - val_loss: 0.7071
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.4953 - loss: 0.6967 - val_accuracy: 0.4400 - val_loss: 0.7120
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.5216 - loss: 0.6935 - val_accuracy: 0.4400 - val_loss: 0.7068
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.4632 - loss: 0.6921 - val_accuracy: 0.5600 - val_loss: 0.6938
Epoch 9/10
4/4

✅ Model saved as breast_cancer_model.h5


In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# ==== CONFIG ====
# ==== CONFIG ====
IMG_DIR = r"C:/Users/Timothy/OneDrive/Desktop/jpeg"  # Your new images folder
MODEL_PATH = r"C:/Users/Timothy/OneDrive/Desktop/breast_cancer_model/breast_cancer_model.h5"  # Correct path to the model
IMAGE_SIZE = 256
OUTPUT_CSV = "predictions.csv"
# ===============

# ===============

# Load the trained model
model = tf.keras.models.load_model(MODEL_PATH)

# Get all image files from the folder
image_files = [f for f in os.listdir(IMG_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

results = []

# Loop through each image and make predictions
for fname in image_files:
    img_path = os.path.join(IMG_DIR, fname)
    img = image.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize the image
    pred = model.predict(img_array)[0][0]  # Model prediction
    label = "malignant" if pred > 0.5 else "benign"  # Classify based on prediction
    results.append((fname, label))
    print(f"{fname} → {label} ({pred:.2f})")

# Save the results to a CSV file
df = pd.DataFrame(results, columns=["filename", "label"])
df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Predictions saved to: {OUTPUT_CSV}")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'C:/Users/Timothy/OneDrive/Desktop/breast_cancer_model/breast_cancer_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)